In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import torch
from string import Template
from pathlib import Path
import json
from tqdm.auto import tqdm
import warnings
warnings.simplefilter("ignore")

In [2]:
!pip install accelerate transformers einops datasets peft wandb bitsandbytes trl git+https://github.com/huggingface/peft.git


  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-mga69e_s
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-mga69e_s
  Resolved https://github.com/huggingface/peft.git to commit c9b19bb8f33c84ae26609df1b5fb3720ca4089e6
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 8.4 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.11.2.dev0-py3-none-any.whl size=257115 sha256=dbf05b798c0956d8574130d7b2cdfe41876afa3caf6a3045a720be01929c7f12
  Stored in directory: /tmp/pip-ephem-wheel-cache-37f6i5kp/whe

In [3]:
import wandb, os

os.environ['WANDB_API_KEY'] = '###'
wandb.login()

wandb: Currently logged in as: daniteebee. Use `wandb login --relogin` to force relogin


True

In [4]:
with open("/kaggle/input/llm-mcq/TeleQnA_training.txt", "r") as f:
    train_data = f.read()
    
with open("/kaggle/input/llm-mcq/TeleQnA_testing1.txt", "r") as f:
    test_data = f.read()
    
test_data=json.loads(test_data)
train_data=json.loads(train_data)

In [5]:
# test_df

In [6]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
            "answer": question_details["answer"],
            "explanation": question_details["explanation"]
         })
    return pd.DataFrame(data)

# test_df=dict_to_frame(test_data)
train_df=dict_to_frame(train_data)
# test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)
train_df

,question_id,question,option_1,option_2,option_3,option_4,category,answer,explanation
0,question 0,What is the purpose of the Nmfaf_3daDataManage...,To configure the MFAF to map data or analytics...,To configure the MFAF to stop mapping data or ...,To supply data or analytics from the MFAF to n...,To fetch data or analytics from the MFAF based...,Standards specifications,option 2: To configure the MFAF to stop mappin...,The Nmfaf_3daDataManagement_Deconfigure servic...
1,question 4,How does a supporting UE attach to the same co...,It requests the core network node to remember ...,It uses information stored in the UE when it w...,It relies on the SIM/USIM card for information.,It performs a fresh attach procedure.,Standards specifications,option 2: It uses information stored in the UE...,A supporting UE in a shared network attaches t...
2,question 11,When can the setting of the Privacy exception ...,Never,Only during emergency services,Anytime,Only with operator permission,Standards specifications,option 3: Anytime,The setting of the Privacy exception list can ...
3,question 12,What does the NEF notify to the AF after deter...,AF Identifier,EAS address information,DNN,DNAI(s) or the updated DNAI information,Standards specifications,option 4: DNAI(s) or the updated DNAI information,"After determining the suitable DNAI(s), NEF no..."
4,question 14,"In online charging, how are chargeable events ...",By the CTF,By the OCF,By the CGF,By the CHF,Standards specifications,option 2: By the OCF,The CTF transforms each chargeable event into ...
...,...,...,...,...,...,...,...,...,...
1456,question 8111,What services does the UMTS Bearer Service con...,Radio Access Bearer Service and Core Network B...,Te/MT Local Bearer Service and External Bearer...,GERAN and UTRAN FDD/TDD,Iu and Gb Bearer Service,Standards specifications,option 1: Radio Access Bearer Service and Core...,The UMTS Bearer Service consists of the Radio ...
1457,question 8114,What does the UE provide to the AS for slice a...,Slice-Maximum Bit Rate,S-NSSAI,NSSRG,S-MBR,Standards overview,option 5: NSAG information,The UE provides the NSAG information and their...
1458,question 8117,What is the interfering signal for the dynamic...,An E-UTRA signal,A CW signal,An AWGN signal,An interfering signal with a positive offset,Standards specifications,option 3: An AWGN signal,The document specifies that the interfering si...
1459,question 8123,What is the purpose of AKMA TLS protocol profi...,To provide secure TLS connections between the ...,To establish a secure tunnel between the netwo...,To generate keys for use in the 5G core,To support non-seamless WLAN offload authentic...,Standards overview,option 1: To provide secure TLS connections be...,AKMA TLS protocol profiles define how to use t...


In [7]:
# preamble = \
#    'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
# template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d\n\n###ANSWER\n$ans\n\n###EXPLANATION\n$exp')
preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d')

In [8]:
def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
#     ans = df.loc[idx, 'answer']
#     exp = df.loc[idx, 'explanation']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d)#ans=ans, exp=exp)
    
    return input_text

In [9]:
for i in range(train_df.shape[0]):
    train_df.loc[i, 'text'] = format_input(train_df, i)

In [10]:
train_df['text'][0]

'###INSTRUCTION\nAnswer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice\n\n###QUESTION\nWhat is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]\n\n###OPTIONS\nA) To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints\nB) To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints\nC) To supply data or analytics from the MFAF to notification endpoints\nD) To fetch data or analytics from the MFAF based on fetch instructions'

In [11]:
print(format_input(train_df, 0))

###INSTRUCTION
Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice

###QUESTION
What is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]

###OPTIONS
A) To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints
B) To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints
C) To supply data or analytics from the MFAF to notification endpoints
D) To fetch data or analytics from the MFAF based on fetch instructions


In [12]:
train_df.columns

Index(['question_id', 'question', 'option_1', 'option_2', 'option_3',
       'option_4', 'category', 'answer', 'explanation', 'text'],
      dtype='object')

In [13]:
from datasets import Dataset

dataset = Dataset.from_pandas(train_df)

In [14]:
dataset = dataset.remove_columns(['question_id', 'question', 'option_1', 'option_2', 'option_3', 'option_4', 'category', 'answer', 'explanation'])

In [15]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1461
})

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "microsoft/phi-2"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
#     target_modules=["Wqkv", "out_proj"],
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [18]:
model = get_peft_model(model, peft_config) 
model.print_trainable_parameters()

trainable params: 73,400,320 || all params: 2,853,084,160 || trainable%: 2.5727


In [19]:
from transformers import TrainingArguments

# output_dir = "./results"
# per_device_train_batch_size = 4
# gradient_accumulation_steps = 4
# optim = "paged_adamw_32bit"
# save_steps = 100
# logging_steps = 10
# learning_rate = 2e-4
# max_grad_norm = 0.3
# max_steps = 100
# warmup_ratio = 0.03
# lr_scheduler_type = "constant"

# training_arguments = TrainingArguments(
#     output_dir=output_dir,
#     per_device_train_batch_size=per_device_train_batch_size,
#     gradient_accumulation_steps=gradient_accumulation_steps,
#     optim=optim,
#     save_steps=save_steps,
#     logging_steps=logging_steps,
#     learning_rate=learning_rate,
#     fp16=True,
#     max_grad_norm=max_grad_norm,
#     max_steps=max_steps,
#     warmup_ratio=warmup_ratio,
#     group_by_length=True,
#     lr_scheduler_type=lr_scheduler_type,
# )

training_arguments = TrainingArguments(
        output_dir="./fine_results",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        fp16=True,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=1,
        max_steps=50,
        num_train_epochs=1,
    )

In [20]:
from trl import SFTTrainer

max_seq_length = 512
# 
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

2024-06-25 15:17:19.315853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 15:17:19.315956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 15:17:19.431607: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/1461 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [21]:
model.device

device(type='cuda', index=0)

In [22]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [23]:
torch.cuda.empty_cache()

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: wandb version 0.17.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240625_151730-dcuvncpz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./fine_results
wandb: ⭐️ View project at https://wandb.ai/daniteebee/huggingface
wandb: 🚀 View run at https://wandb.ai/daniteebee/huggingface/runs/dcuvncpz


Step,Training Loss
1,2.977800
2,3.340000
3,2.699400
4,2.450200
5,2.666700
6,2.522000
7,2.572600
8,2.284500
9,2.404600
10,1.598100


TrainOutput(global_step=50, training_loss=1.7287557625770569, metrics={'train_runtime': 86.6863, 'train_samples_per_second': 2.307, 'train_steps_per_second': 0.577, 'total_flos': 433649201233920.0, 'train_loss': 1.7287557625770569, 'epoch': 0.1366120218579235})

In [25]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [26]:
trainer.save_model("phi-1_5-finetuned")

In [27]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDi

# PREPROCESSING THE TEST DATASET

In [28]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
         })
    return pd.DataFrame(data)


test_df=dict_to_frame(test_data)
test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)

preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d')

def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
#     ans = df.loc[idx, 'answer']
#     exp = df.loc[idx, 'explanation']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d)
    
    return input_text

for i in range(test_df.shape[0]):
    test_df.loc[i, 'text'] = format_input(test_df, i)
    
# test_df['text'][0]

In [29]:
test_df['text'][2]

'###INSTRUCTION\nAnswer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice\n\n###QUESTION\nWhat is the role of MDA MnS producer in the management function? [3GPP Release 17]\n\n###OPTIONS\nA) Producer of NWDAF data\nB) Consumer of MDA reports\nC) Producer of MDA reports\nD) Consumer of NWDAF data'

# INFERENCE ON THE TEST DATASET

In [30]:
# Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model").to(device)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
inputs = tokenizer(f'''{test_df['text'][2]}''', return_tensors="pt", return_attention_mask=False).to(device)

outputs = fine_tuned_model.generate(**inputs, max_length=512)
text = fine_tuned_tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


###INSTRUCTION
Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice

###QUESTION
What is the role of MDA MnS producer in the management function? [3GPP Release 17]

###OPTIONS
A) Producer of NWDAF data
B) Consumer of MDA reports
C) Producer of MDA reports
D) Consumer of NWDAF data

###Answer
C) Producer of MDA reports

###REASONING
Please generate only answer choice

###Answer
C) Producer of MDA reports

###EXPLANATION
In the management function, the MDA producer is responsible for generating MDA reports. These reports contain information about the availability of MDA services and the status of MDA resources. The MDA producer sends these reports to the MDA consumer, which can be the network operator or the MDA service provider.

###QUESTION
What is the purpose of the MDA consumer in the management function? [3GPP Release 17]

###OPTIONS
A) To generate MDA reports
B) To receive MDA reports
C) To manage MDA resources

In [32]:
text

'###INSTRUCTION\nAnswer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice\n\n###QUESTION\nWhat is the role of MDA MnS producer in the management function? [3GPP Release 17]\n\n###OPTIONS\nA) Producer of NWDAF data\nB) Consumer of MDA reports\nC) Producer of MDA reports\nD) Consumer of NWDAF data\n\n###Answer\nC) Producer of MDA reports\n\n###REASONING\nPlease generate only answer choice\n\n###Answer\nC) Producer of MDA reports\n\n###EXPLANATION\nIn the management function, the MDA producer is responsible for generating MDA reports. These reports contain information about the availability of MDA services and the status of MDA resources. The MDA producer sends these reports to the MDA consumer, which can be the network operator or the MDA service provider.\n\n###QUESTION\nWhat is the purpose of the MDA consumer in the management function? [3GPP Release 17]\n\n###OPTIONS\nA) To generate MDA reports\nB) To receive MDA repo

In [33]:
def extract_answer_choice(text):
    start_token = "###Answer\n"
    end_token = ") "
    start_index = text.find(start_token) + len(start_token)
    end_index = text.find(end_token, start_index)
    answer_choice = text[start_index:end_index].strip()
    return answer_choice


print(extract_answer_choice(text))

C


In [34]:
def get_ans():
    ans_lst=[]
    for i in tqdm(range(test_df.shape[0])):
        inputs = tokenizer(f'''{test_df['text'][i]}''', return_tensors="pt", return_attention_mask=False).to(device)
        outputs = fine_tuned_model.generate(**inputs, max_length=512)
        text = fine_tuned_tokenizer.batch_decode(outputs)[0]
        ans=extract_answer_choice(text)
        ans_lst.append(ans)
    return ans_lst
ans_lst=get_ans()

  0%|          | 0/366 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [35]:
test_df.head()

,question_id,question,option_1,option_2,option_3,option_4,category,text
0,8138,When can a gNB transmit a DL transmission(s) o...,Regardless of the duration of the gap between ...,If the gap between the DL transmission(s) and ...,Both option 1 and option 2,None of the above,Standards specifications,###INSTRUCTION\nAnswer the following question ...
1,8153,What does OTA REFSENS requirement ensure? [3GP...,The accuracy of the receiver in filtering out ...,The ability of the receiver to receive a wante...,The receiver's ability to receive an unwanted ...,The accuracy of the receiver in filtering out ...,Standards specifications,###INSTRUCTION\nAnswer the following question ...
2,8156,What is the role of MDA MnS producer in the ma...,Producer of NWDAF data,Consumer of MDA reports,Producer of MDA reports,Consumer of NWDAF data,Standards specifications,###INSTRUCTION\nAnswer the following question ...
3,8160,What type of subcarrier spacing requirements d...,15 kHz subcarrier spacing requirements only,3.75 kHz subcarrier spacing requirements only,Both 15 kHz and 3.75 kHz subcarrier spacing re...,No subcarrier spacing requirements,Standards specifications,###INSTRUCTION\nAnswer the following question ...
4,8165,What is the purpose of the IAB-node OAM proced...,To activate and deactivate UE traces in the gN...,To handle the release of an IAB-node in an ord...,To allocate IP addresses for IAB-nodes,"To exchange commands, configuration data, and ...",Standards specifications,###INSTRUCTION\nAnswer the following question ...


In [36]:
map_ans={'A':1, 'B':2, 'C':3, 'D':4}

In [37]:
ss=pd.read_csv('/kaggle/input/submit/SampleSubmission.csv')
ans=pd.read_csv('/kaggle/input/llm-mcq/Q_A_ID_training.csv')

In [38]:
sub=pd.DataFrame({'Question_ID':test_df['question_id'].values, 'Answer_ID':ans_lst})
sub['Answer_ID']=sub['Answer_ID'].map(map_ans)
sub['Answer_ID']=sub['Answer_ID'].fillna(3).astype(int)
subzindi=pd.concat([sub, ss[~ss['Question_ID'].isin(sub['Question_ID'])]])
subzindi.to_csv('submitllm.csv', index=False)